In [1]:
# pip install psycopg2 если библиотека не установлена
import psycopg2
import pandas as pd

In [2]:
database="startml",
user="robot-startml-ro",
password="pheiph0hahj1Vaif",
host="postgres.lab.karpov.courses",
port=6432

In [3]:
'''
Сделайте запрос на все колонки из таблицы user. Для этого может понадобиться взять user в кавычки, вот так: 
"user" - чтобы PostgreSQL не спутал ее со своей встроенной таблицей user, где он ведет учет всех пользователей.
Напишите через пробел, с какого id начинаются записи и какой город у самого первого ID (на английском)
'''
df = pd.read_sql(
    """SELECT * FROM "user"  """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df.head(1)

,id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads


In [4]:
'''
Отберите уникальные значения тем (topic) в таблице постов (post).
Напишите через пробел количество записей и самое длинное название топика.
'''
df = pd.read_sql(
    """SELECT DISTINCT topic FROM "post"  """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,topic
0,tech
1,covid
2,movie
3,entertainment
4,politics
5,sport
6,business


In [5]:
'''
Отберите пользователей старше 30 с устройством на iOS. 
Каков ID и город записи с минимальным ID (он идет первым в таблице)? Запишите значения через пробел.
'''
df = pd.read_sql(
    """
    SELECT id, city 
    FROM "user"  
    WHERE age>30 AND os = 'iOS'
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df.head(1)

,id,city
0,212,Podolsk


In [6]:
'''
Отберите пользователей, которые не из России и при этом либо их экспериментальная группа не 0 и не 3 или их город – Минск.

Напишите через пробел первые 4 ID в такой выдаче.
'''
df = pd.read_sql(
    """
    SELECT id, city 
    FROM "user"  
    WHERE country != 'Russia' AND (city = 'Minsk' AND os = 'iOS' OR exp_group NOT IN (0,3))
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df.head(4)

,id,city
0,211,Minsk
1,213,Kamianske
2,216,Lviv
3,218,Minsk


In [7]:
'''
Подсчитайте средний возраст пользователей в разрезе страны. 
В ответ напишите средний возраст пользователей из Кипра. 
Ответ округлите до двух знаков после точки.
'''
df = pd.read_sql(
    """
    SELECT ROUND(AVG(age),2) AS avg_age_cyprus
    FROM "user"  
    WHERE country = 'Cyprus'
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,avg_age_cyprus
0,28.07


In [8]:
'''
Сгруппируйте пользователей сначала по экспериментальной группе, а затем по операционной системе. 
В каждой группе подсчитайте количество пользователей, максимальный и минимальный возраст.

Сохраните результаты в файл CSV колонками exp_group, os, total_users, max_age, min_age (обратите внимание на название последних трех колонок!), 
используя запятую в качестве сепаратора, и отправьте файл в LMS.

Для сохранения в CSV в Redash внизу есть кнопка "три точки", где выпадает пункт "Download as CSV file".
'''
df = pd.read_sql(
    """
    select exp_group, 
        os, 
        COUNT(user) as total_users, 
        MAX(age) as max_age, 
        MIN(age) as min_age
    from "user"
    group by exp_group, os
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

#df.to_csv('les6_step8.csv',sep=';',index=False)

In [9]:
'''
В каждой категории постов определите длину самого длинного текста, используя таблицу post. В ответ отправьте тему, у которой длина самого большого поста больше 25 000.
'''
df = pd.read_sql(
    """
    SELECT topic, LENGTH(text)
    FROM "post"
    WHERE LENGTH(text)>25000
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,topic,length
0,politics,25392


In [10]:
'''
Найдите страну с наименьшим числом пользователей среди стран, у которых общее число пользователей больше 1000. Введите ее название на английском.
'''
df = pd.read_sql(
    """
    SELECT country, COUNT(user)
    FROM "user"
    GROUP BY country
    HAVING COUNT(user)>1000
    ORDER BY COUNT(user)
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,country,count
0,Azerbaijan,1542
1,Finland,1599
2,Turkey,1606
3,Kazakhstan,3172
4,Belarus,3293
5,Ukraine,8273
6,Russia,143035


In [11]:
'''
Что выведет следующий запрос при его применении на таблице user?

SELECT country, exp_group, COUNT(id)
FROM "user"
GROUP BY country, exp_group;
'''
df = pd.read_sql(
    """
    SELECT country, exp_group, COUNT(id)
    FROM "user"
    GROUP BY country, exp_group;
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df.head()

,country,exp_group,count
0,Azerbaijan,0,341
1,Azerbaijan,1,271
2,Azerbaijan,2,327
3,Azerbaijan,3,308
4,Azerbaijan,4,295


In [12]:
'''
Отберите для пользователей из Москвы экспериментальные группы, в которых средний возраст больше 27.2. 
'''
df = pd.read_sql(
    """
    SELECT exp_group, COUNT(user)
    FROM "user"
    WHERE city = 'Moscow'
    GROUP BY country, exp_group
    HAVING AVG(age)>27.2
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,exp_group,count
0,1,4414
1,3,4378


In [13]:
'''
Сейчас мы будем работать с другой таблицей под названием post. 
Найдите в таблице post 3 темы с наибольшим количеством постов. 
'''
df = pd.read_sql(
    """
    SELECT topic, COUNT(id)
    FROM "post"
    GROUP BY topic
    ORDER BY COUNT(id) DESC
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df.head(3)

,topic,count
0,movie,3000
1,covid,1799
2,business,510


In [14]:
'''
Отберите пользователей из Воронежа (Voronezh) и отсортируйте их по возрасту в убывающем порядке. 
Для равных возрастов отсортируйте по возрастанию экспериментальной группы.

Напишите ID второго в порядке выдачи пользователя.
'''
df = pd.read_sql(
    """
    SELECT id,city,age,exp_group
    FROM "user"
    WHERE city = 'Voronezh'
    ORDER BY age DESC, exp_group
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df[1:2]

,id,city,age,exp_group
1,2441,Voronezh,71,3


In [15]:
'''
Отберите post_id, time, age и os пользователя для всех лайков пользователей из Омска. 
Отсортируйте результаты по убыванию времени действия и загрузите в LMS первые 100 значений в формате CSV (сепаратор "запятая", колонки с теми же названиями, что и в задании).
'''
df = pd.read_sql(
    """
    SELECT fa.post_id, fa.time, u.age, u.os
    FROM feed_action fa
    JOIN "user" u
    on fa.user_id = u.id
    where u.city = 'Omsk' AND fa.action = 'like'
    order by fa.time desc
    limit 100
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

#df.to_csv('les6_step15.csv',sep=',')

In [ ]:
'''
Найдите город, пользователи которого в возрасте 36 лет 2021-12-01 числа просмотрели запись на тему covid меньше всего раз относительно других городов. 
Введите его название на английском.
'''
df = pd.read_sql(
    """
    SELECT u.city
    FROM feed_action fa
    JOIN "user" u
    on fa.user_id=u.id
    JOIN post p
    ON fa.post_id=p.id
    where u.age = 36 AND p.topic = 'covid' AND CAST(fa.time as DATE) = '2021-12-01' AND fa.action = 'view'
    group by u.city
    order by count(p.id) asc
    limit 1
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

In [ ]:
'''
Напишите SQL запрос, используя который вы получите:
- топ N постов по лайкам,
- количество пролайкавших пользователей,
- время последнего лайка.

В качестве ответа напишите через пробел первые 3 ID постов из этого топа.

Сохраните запрос, он может понадобиться в будущем в финальном проекте.
'''
df = pd.read_sql(
    """
    SELECT post.id, COUNT(feed_action.user_id) AS likes_count, MAX(feed_action.time) AS last_like_time
    FROM post
    LEFT JOIN feed_action ON post.id = feed_action.post_id AND feed_action.action = 'like'
    GROUP BY post.id
    ORDER BY likes_count DESC
    LIMIT 3 --N;
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df